<a href="https://colab.research.google.com/github/MichelledKGit/Michelle_PhD_Chap3/blob/main/Testing_Fuzzy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
!git clone https://github.com/MichelledKGit/Michelle_PhD_Chap3.git

Cloning into 'Michelle_PhD_Chap3'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 39 (delta 17), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (39/39), 25.76 KiB | 8.59 MiB/s, done.
Resolving deltas: 100% (17/17), done.


# Python code for Chapter 3: SEASONAL CATCHMENT AREAS USING AN ATTRIBUTE BASED FUZZY LATTICE DATA STRUCTURE for toy example. Please note that the data used in POI catchment areas is not publically available and cannot be uploaded in this repository. Data ethics clearance number NAS003/2023.


# Importing all the relevant packages

In [61]:
import pandas as pd
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
import networkx as nx
import os
from functools import reduce
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# Enable inline plotting
%matplotlib inline

# NB: The names in the structured nodes (1-9) are in the ordered format for V that the labelled states are first  i.e. grid_name={1,2,3,4,5,6,7,8,9}={v2,v7,v1,v3,v4,v5,v6,v8,v9}


# Import Datasets

In [62]:
#Import Grid_Neighbours_Labels.
# Grid_Name and Grid_Name2 indicates the the structural nodes adjacent to each other.
# Absorbant_State indicates if grid_name is absorbant(1) or not and same for Absorbant_State2 for grid_name2.
# Label_Val indicates the label associated with Grid_Name where for the toy example, P1 and P1 indicates the two POIs and U3-U9 indicates unlabelled.
base_path = 'Michelle_PhD_Chap3'
file_path = os.path.join(base_path, 'Grid_Neighbours_Labels.txt')

DF = pd.read_csv(file_path, sep='|')
print(DF.head())

   Grid_Name  Grid_Name2  Absorbant_State  Absorbant_State2 Label_val
0          1           1                1                 1        P1
1          3           1                0                 1        U3
2          4           1                0                 1        U4
3          6           1                0                 1        U6
4          2           2                1                 1        P2


In [63]:
# Gives the attributes per grid. Each additional attribute will be added in another column the code is dynically set-up that way to pick up all distinct levels in all attributes provided
#For toy example there is one attribute with two levels the level a2 is associated with structural nodes v1 and v7 and level a1 with v2,v3,v4,v5,v6,v8 and v9.
file_path1 = os.path.join(base_path, 'Grid_Attributes.txt')

Attributes = pd.read_csv(file_path1, sep='|')
Attributes = Attributes.sort_values(by='grid_name')
print(Attributes.head())

   grid_name attribute1
0          1         A1
1          2         A2
2          3         A2
3          4         A1
4          5         A1


In [64]:
# Degrees File. Distinct on grid_name and give the total number of neighbouring structural nodes(total) and if it is an absorbent state
file_path2 = os.path.join(base_path, 'Grid_Degrees.txt')

# Read the file
DF2 = pd.read_csv(file_path2, sep='|')
print(DF2.head())

absorbant_States=sum(DF2.Absorbant_State)
nodes=len(DF2.index)

   grid_Name  Absorbant_State  Total
0          1                1      3
1          2                1      2
2          3                0      2
3          4                0      2
4          5                0      3


# Create Adjacency Matrix

In [65]:
#Using the information from DF create adjacency matrix
df = pd.crosstab(DF.Grid_Name, DF.Grid_Name2)
idx = df.columns.union(df.index)
df = df.reindex(index = idx, columns=idx, fill_value=0)

#Create correct format in Links table (new adjacency matrix)
links = np.matrix(df).astype(np.float64)
identity_abs = np.identity(absorbant_States)
#Absorbent states should only indicate 1 to own state, nullify absorbent states and replace top left hand corner of matrix with identity matrix
links[0:absorbant_States]=0
#Replace the identity matrix in the top left hand corner of links matrix that all absorbent states can only move into themselves
links[0:absorbant_States,0:absorbant_States]=identity_abs

#Links is now the adjacency matrix but all absorbent states only has a probability of 1 to stay in the same state with the rest 0
print(links)

[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 1. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0.]]


# Import the attributes and create attribute matrix X with N the number of structural nodes, A the total number of attributes and T total number of unique levels over all attributes

In [66]:
# Step 1: Order by grid_name
Attributes_sorted = Attributes.sort_values(by='grid_name')

# Step 2: Drop grid_name
Attributes_dropped = Attributes_sorted.drop(columns=['grid_name'])

# Step 3: Stack remaining columns into attributes_raw
attributes_raw = np.column_stack([Attributes_dropped[col].values for col in Attributes_dropped.columns])

# Convert the attributes_raw matrix to a DataFrame
df_attributes = pd.DataFrame(attributes_raw)
df_attributes.columns = df_attributes.columns.astype(str)

# Count the number of attributes (M)
A = df_attributes.shape[1]
N = links.shape[1]

# Extract all unique values across all attributes
distinct_values = np.unique(attributes_raw)

# Count the number of distinct values in all attributes (ki)
T = len(distinct_values)

# Initialize OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform the categorical variables
attributes_encoded = encoder.fit_transform(df_attributes).toarray()

# Extract feature names from the encoder
feature_names = encoder.get_feature_names_out(df_attributes.columns)

# Create a new DataFrame with one-hot encoded columns
df_encoded = pd.DataFrame(attributes_encoded, columns=feature_names)

# Attribute matrix X
X = df_encoded.values

X = X.astype(np.float64)

print(N)
print(A)
print(T)

print(X)

9
1
2
[[1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [67]:
#Structural weights, if all is 1 it it equal to the adjacency matrix
w0=links
# Create w with the weights of the attributes
w = np.full(A, 1)

#Shell for transition matrix
transition_matrix = np.zeros((N + T, N + T))

#Sum over the rows of structural weights w0
row_sums = w0.sum(axis=1)
row_sums=row_sums.reshape((N, 1))

#sum over the rows of the attribute weights
attributes_sums = X.T.sum(axis=1)
attributes_sums=attributes_sums.reshape((T, 1))

# Create shell for submatrices with zeros
Matrix_Ps = np.zeros((N, N))
Matrix_Psa_Structure = X
Matrix_Pas=np.zeros((T,N))
Matrix_Pas_Structure = X.T
Matrix_0 = np.zeros((T, T))


# Create a dictionary to map attribute values to their corresponding weights
weights_dict = {}
for i in range(A):
    col_values = np.unique(df_attributes.iloc[:, i])
    for val in col_values:
        weights_dict[val] = w[i]

# Create the row vector w with the corresponding weights
wj = np.array([weights_dict[val] for val in distinct_values])
sum_weights=np.sum(w)

#From equations the values for submatrices
Matrix_Ps=w0/(row_sums+sum_weights)

Matrix_A=wj/(row_sums+sum_weights)
Matrix_A = np.asarray(Matrix_A)
Matrix_A *= Matrix_Psa_Structure

Matrix_Psa_Structure = np.asarray(Matrix_Psa_Structure)

Matrix_Pas=(1/attributes_sums) * Matrix_Pas_Structure

transition_matrix=np.vstack((np.hstack((Matrix_Ps,Matrix_A)),np.hstack((Matrix_Pas,Matrix_0))))
transition_matrix = np.round(transition_matrix, 6)

#The Transition matrix shown in the article has the order v1,v2,v3,...,v9,va1,va2 to illustrate the probabilities in each row without confusing the reader and is also Pa before introducing absorbent states.
#The structure in the code is different with the absorbant states begin first i.e row 1 and column 1 is node v2 and row 2 and column 2 is node V7 as this matrix was set up for the label propagation
print(transition_matrix)

[[0.5      0.       0.       0.       0.       0.       0.       0.
  0.       0.5      0.      ]
 [0.       0.5      0.       0.       0.       0.       0.       0.
  0.       0.       0.5     ]
 [0.333333 0.       0.       0.       0.333333 0.       0.       0.
  0.       0.       0.333333]
 [0.333333 0.       0.       0.       0.       0.       0.333333 0.
  0.       0.333333 0.      ]
 [0.       0.25     0.25     0.       0.       0.25     0.       0.
  0.       0.25     0.      ]
 [0.2      0.       0.       0.       0.2      0.       0.2      0.2
  0.       0.2      0.      ]
 [0.       0.       0.       0.25     0.       0.25     0.       0.
  0.25     0.25     0.      ]
 [0.       0.25     0.       0.       0.       0.25     0.       0.
  0.25     0.25     0.      ]
 [0.       0.       0.       0.       0.       0.       0.333333 0.333333
  0.       0.333333 0.      ]
 [0.142857 0.       0.       0.142857 0.142857 0.142857 0.142857 0.142857
  0.142857 0.       0.      ]
 [0.   

# Check if matrix is stochastic

In [68]:
def is_row_stochastic(matrix):
    # Check non-negativity
    if np.any(matrix < 0):
        return False
    # Check if each row sum is within the range [0.99, 1.01]
    row_sums = matrix.sum(axis=1)
    return np.all((row_sums >= 0.99) & (row_sums <= 1.01))

# Example usage:
print("Is row stochastic?", is_row_stochastic(transition_matrix))

Is row stochastic? True


In [69]:
# Partition the transition matrix into blocks as indicated on article
P_ll = transition_matrix[:absorbant_States, :absorbant_States]
P_ul = transition_matrix[absorbant_States:, :absorbant_States]
P_uu = transition_matrix[absorbant_States:, absorbant_States:]

print(P_ll)
print(P_ul)
print(P_uu)

[[0.5 0. ]
 [0.  0.5]]
[[0.333333 0.      ]
 [0.333333 0.      ]
 [0.       0.25    ]
 [0.2      0.      ]
 [0.       0.      ]
 [0.       0.25    ]
 [0.       0.      ]
 [0.142857 0.      ]
 [0.       0.5     ]]
[[0.       0.       0.333333 0.       0.       0.       0.       0.
  0.333333]
 [0.       0.       0.       0.       0.333333 0.       0.       0.333333
  0.      ]
 [0.25     0.       0.       0.25     0.       0.       0.       0.25
  0.      ]
 [0.       0.       0.2      0.       0.2      0.2      0.       0.2
  0.      ]
 [0.       0.25     0.       0.25     0.       0.       0.25     0.25
  0.      ]
 [0.       0.       0.       0.25     0.       0.       0.25     0.25
  0.      ]
 [0.       0.       0.       0.       0.333333 0.333333 0.       0.333333
  0.      ]
 [0.       0.142857 0.142857 0.142857 0.142857 0.142857 0.142857 0.
  0.      ]
 [0.5      0.       0.       0.       0.       0.       0.       0.
  0.      ]]


In [70]:
# Calculate the stationary distribution for the unlabeled nodes
P_inf = inv(np.eye(P_uu.shape[0]) - P_uu).dot(P_ul)

# Define the labels for the labeled nodes
Y_l = np.identity(absorbant_States)  # Labels for node 1 and node 2

# Calculate the labels for the unlabeled nodes
Y_u = P_inf.dot(Y_l)

# Combine the results for all nodes
Y_all = np.vstack([Y_l, Y_u])

# Print the final probability matrix
print("Final probability matrix:")
print(Y_all)

Final probability matrix:
[[1.         0.        ]
 [0.         1.        ]
 [0.59390069 0.40609736]
 [0.78833178 0.21166501]
 [0.4847535  0.51524461]
 [0.66813817 0.33185965]
 [0.68802257 0.31197424]
 [0.49093962 0.50905798]
 [0.61864516 0.38135084]
 [0.67697515 0.32302144]
 [0.29695034 0.70304868]]


In [71]:
# Example usage:
print("Is row stochastic?", is_row_stochastic(Y_all))

Is row stochastic? True


# Drivetimes: Please note for the toy example the same nodes with be indicated for drive-times 5, 10 and 15minutes but for real-world applications like the pharmacy sales more nodes will be included as the drive times extend

In [72]:
# Only take the first N rows as only the strctural nodes can be used for drivetimes
L=Y_all[:N]

In [73]:
# The Grid_Name column indicates the grid that is allocated in the drive time circle of Store_Grid. I.e. grid names 1 and 2 are the only grids with a POI and which have a drivetime circle associated with it.
file_path3 = os.path.join(base_path, 'Grids_to_Drivetimes_5min.txt')

# Read the file
DT5 = pd.read_csv(file_path3, sep='|')
print(DT5)

   grid_name  store_grid
0          1           1
1          4           1
2          6           1
3          7           1
4          2           2
5          5           2
6          6           2
7          8           2


In [74]:
# The Grid_Name column indicates the grid that is allocated in the drive time circle of Store_Grid. I.e. grid names 1 and 2 are the only grids with a POI and which have a drivetime circle associated with it.
file_path4 = os.path.join(base_path, 'Grids_to_Drivetimes_10min.txt')

# Read the file
DT10 = pd.read_csv(file_path4, sep='|')
print(DT10)

   grid_name  store_grid
0          1           1
1          4           1
2          6           1
3          7           1
4          2           2
5          5           2
6          6           2
7          8           2


In [75]:
# The Grid_Name column indicates the grid that is allocated in the drive time circle of Store_Grid. I.e. grid names 1 and 2 are the only grids with a POI and which have a drivetime circle associated with it.
file_path5 = os.path.join(base_path, 'Grids_to_Drivetimes_15min.txt')

# Read the file
DT15 = pd.read_csv(file_path5, sep='|')
print(DT15)

   grid_name  store_grid
0          1           1
1          4           1
2          6           1
3          7           1
4          2           2
5          5           2
6          6           2
7          8           2


In [76]:
#5 minute drive time mappings
#Create and indicator matrix with the columns the POIs and the rows indicating which nodes are in the drive-times of those POIs
n_cols, n_rows = absorbant_States+1, nodes+1
Dist5 = np.zeros((n_rows, n_cols), dtype=np.int32)

Dist5[0,:] = np.arange(n_cols)
Dist5[:,0] = np.arange(n_rows)

d_selection5 = pd.DataFrame(DT5, columns = ['grid_name','store_grid'])

for i in range(len(d_selection5)):
    Dist5[d_selection5.loc[i, "grid_name"],d_selection5.loc[i, "store_grid"]]=1

Dist5=pd.DataFrame(Dist5)
Dist5 = Dist5.iloc[1: , 1:]
Dist5=pd.DataFrame(Dist5)

myArray5 = np.where(Dist5==0, Dist5, L)
myArray5[0:absorbant_States,0:absorbant_States]=identity_abs
res5 = myArray5/myArray5.sum(axis=1)[:,None]
res5=pd.DataFrame(res5)
res5=res5.fillna(0)

# Add the grid_name column from DT5
Dist5['grid_name'] = Attributes['grid_name'].values
print(Dist5)

   1  2  grid_name
1  1  0          1
2  0  1          2
3  0  0          3
4  1  0          4
5  0  1          5
6  1  1          6
7  1  0          7
8  0  1          8
9  0  0          9


<ipython-input-76-ca5c8fc231d1>:20: RuntimeWarning: invalid value encountered in divide
  res5 = myArray5/myArray5.sum(axis=1)[:,None]


In [77]:
#10 minute drive time mappings
#Create and indicator matrix with the columns the POIs and the rows indicating which nodes are in the drive-times of those POIs
n_cols, n_rows = absorbant_States+1, nodes+1
Dist10 = np.zeros((n_rows, n_cols), dtype=np.int32)

Dist10[0,:] = np.arange(n_cols)
Dist10[:,0] = np.arange(n_rows)

d_selection10 = pd.DataFrame(DT10, columns = ['grid_name','store_grid'])

for i in range(len(d_selection10)):
    Dist10[d_selection10.loc[i, "grid_name"],d_selection10.loc[i, "store_grid"]]=1

Dist10=pd.DataFrame(Dist10)
Dist10 = Dist10.iloc[1: , 1:]
Dist10=pd.DataFrame(Dist10)

myArray10 = np.where(Dist10==0, Dist10, L)
myArray10[0:absorbant_States,0:absorbant_States]=identity_abs
res10 = myArray10/myArray10.sum(axis=1)[:,None]
res10=pd.DataFrame(res10)
res10=res10.fillna(0)

# Add the grid_name column from DT5
Dist10['grid_name'] = Attributes['grid_name'].values
print(Dist10)

   1  2  grid_name
1  1  0          1
2  0  1          2
3  0  0          3
4  1  0          4
5  0  1          5
6  1  1          6
7  1  0          7
8  0  1          8
9  0  0          9


<ipython-input-77-59d426be79e4>:20: RuntimeWarning: invalid value encountered in divide
  res10 = myArray10/myArray10.sum(axis=1)[:,None]


In [78]:
#15 minute drive time mappings
#Create and indicator matrix with the columns the POIs and the rows indicating which nodes are in the drive-times of those POIs
n_cols, n_rows = absorbant_States+1, nodes+1
Dist15 = np.zeros((n_rows, n_cols), dtype=np.int32)

Dist15[0,:] = np.arange(n_cols)
Dist15[:,0] = np.arange(n_rows)

d_selection15 = pd.DataFrame(DT15, columns = ['grid_name','store_grid'])

for i in range(len(d_selection15)):
    Dist15[d_selection15.loc[i, "grid_name"],d_selection15.loc[i, "store_grid"]]=1

Dist15=pd.DataFrame(Dist15)
Dist15 = Dist15.iloc[1: , 1:]
Dist15=pd.DataFrame(Dist15)

myArray15 = np.where(Dist15==0, Dist15, L)
myArray15[0:absorbant_States,0:absorbant_States]=identity_abs
res15 = myArray10/myArray15.sum(axis=1)[:,None]
res15=pd.DataFrame(res15)
res15=res15.fillna(0)

# Add the grid_name column from DT5
Dist15['grid_name'] = Attributes['grid_name'].values
print(Dist15)

   1  2  grid_name
1  1  0          1
2  0  1          2
3  0  0          3
4  1  0          4
5  0  1          5
6  1  1          6
7  1  0          7
8  0  1          8
9  0  0          9


<ipython-input-78-34d07bcc4c6d>:20: RuntimeWarning: invalid value encountered in divide
  res15 = myArray10/myArray15.sum(axis=1)[:,None]


# Supply and demand to calculate supply-demand ratio and accessibility

In [79]:
# The Grid_Name column indicates the grid that is allocated in the drive time circle of Store_Grid. I.e. grid names 1 and 2 are the only grids with a POI and which have a drivetime circle associated with it.
#The Demand for each grid in the toy example is 1
file_path6 = os.path.join(base_path, 'Grid_Demand.txt')

# Read the file
Grid_Demand = pd.read_csv(file_path6, sep='|')

# Convert grid_name to numeric if it's stored as string
Grid_Demand['grid_name'] = Grid_Demand['grid_name'].astype(int)
# Now sort
Grid_Demand.sort_values('grid_name', inplace=True)
Grid_Demand.reset_index(drop=True, inplace=True)
print(Grid_Demand)

   grid_name  Total
0          1      1
1          2      1
2          3      1
3          4      1
4          5      1
5          6      1
6          7      1
7          8      1
8          9      1


In [80]:
# The Grid_Name column indicates the grid that is allocated in the drive time circle of Store_Grid. I.e. grid names 1 and 2 are the only grids with a POI and which have a drivetime circle associated with it.
# The supply for the toy example for P1 and P2 is 4
file_path7 = os.path.join(base_path, 'Grid_Supply.txt')

# Read the file
Grid_Supply = pd.read_csv(file_path7, sep='|')
print(Grid_Supply)

   grid_name  total_sales POI_Name
0          1            4     POI1
1          2            4     POI2


In [81]:
#Calculate the total demand per grid based on the drive time rings: Demand at 5 minute drive-times
Grid_Demand=pd.DataFrame(Grid_Demand)
Grid_T=Grid_Demand.Total
res5_T=res5.T

Grid_Demand_5min=np.multiply(res5_T,Grid_T)
Grid_Demand_5min=Grid_Demand_5min.T

Total_Grid_Demand_5min=round(Grid_Demand_5min.sum(axis='rows'),2)
Total_Grid_Demand_5min=pd.DataFrame(Total_Grid_Demand_5min)
Total_Grid_Demand_5min.rename(columns={0:'Total_Grid_5min'})
Total_Grid_Demand_5min.insert(0, 'grid_name', range(1, 1 + len(Total_Grid_Demand_5min)))
Total_Grid_Demand_5min.rename(columns={0:'Total_Grid_5min'})

,grid_name,Total_Grid_5min
0,1,3.67
1,2,3.33


In [82]:
#Calculate the total demand per grid based on the drive time rings: Demand at 10 minute drive-times
Grid_Demand=pd.DataFrame(Grid_Demand)
Grid_T=Grid_Demand.Total
res10_T=res10.T

Grid_Demand_10min=np.multiply(res10_T,Grid_T)
Grid_Demand_10min=Grid_Demand_10min.T
Total_Grid_Demand_10min=round(Grid_Demand_10min.sum(axis='rows'),2)
Total_Grid_Demand_10min=pd.DataFrame(Total_Grid_Demand_10min)
Total_Grid_Demand_10min.rename(columns={0:'Total_Grid_10min'})
Total_Grid_Demand_10min.insert(0, 'grid_name', range(1, 1 + len(Total_Grid_Demand_10min)))
Total_Grid_Demand_10min.rename(columns={0:'Total_Grid_10min'})

,grid_name,Total_Grid_10min
0,1,3.67
1,2,3.33


In [83]:
#Calculate the total demand per grid based on the drive time rings: Demand at 15 minute drive-times
Grid_Demand=pd.DataFrame(Grid_Demand)
Grid_T=Grid_Demand.Total
res15_T=res15.T

Grid_Demand_15min=np.multiply(res15_T,Grid_T)
Grid_Demand_15min=Grid_Demand_15min.T
Total_Grid_Demand_15min=round(Grid_Demand_15min.sum(axis='rows'),2)
Total_Grid_Demand_15min=pd.DataFrame(Total_Grid_Demand_15min)
Total_Grid_Demand_15min.rename(columns={0:'Total_Grid_15min'})
Total_Grid_Demand_15min.insert(0, 'grid_name', range(1, 1 + len(Total_Grid_Demand_15min)))
Total_Grid_Demand_15min.rename(columns={0:'Total_Grid_15min'})

,grid_name,Total_Grid_15min
0,1,3.67
1,2,3.33


In [88]:
# Step 2: Manually set correct column names
Grid_Supply.columns = ['grid_name', 'Total_Sales','POI_Name']
Total_Grid_Demand_5min.columns = ['grid_name', 'Total_Demand_5min']
Total_Grid_Demand_10min.columns = ['grid_name', 'Total_Demand_10min']
Total_Grid_Demand_15min.columns = ['grid_name', 'Total_Demand_15min']

# Step 3: Merge cleanly
df_merged = reduce(
    lambda left, right: pd.merge(left, right, on='grid_name', how='outer'),
    [Grid_Supply, Total_Grid_Demand_5min, Total_Grid_Demand_10min, Total_Grid_Demand_15min]
)

df_merged = reduce(
    lambda left, right: pd.merge(left, right, on='grid_name', how='outer'),
    [Grid_Supply, Total_Grid_Demand_5min, Total_Grid_Demand_10min, Total_Grid_Demand_15min]
)
print(df_merged)

   grid_name  Total_Sales POI_Name  Total_Demand_5min  Total_Demand_10min  \
0          1            4     POI1               3.67                3.67   
1          2            4     POI2               3.33                3.33   

   Total_Demand_15min  
0                3.67  
1                3.33  


In [89]:
#Add Supply-Demand Ratio for each drive-time. Since for the toy example all drivetimes are the same, the supply-demand ratio is also the same for 5,10 and 15min
# Replace NaNs with 0
df_merged['Total_Sales'] = df_merged['Total_Sales'].fillna(0)
df_merged['Total_Demand_5min'] = df_merged['Total_Demand_5min'].fillna(0)

# Convert to numeric in case of object type
df_merged['Total_Sales'] = pd.to_numeric(df_merged['Total_Sales'], errors='coerce')
df_merged['Total_Demand_5min'] = pd.to_numeric(df_merged['Total_Demand_5min'], errors='coerce')

# Add new columns
df_merged['AR_5min'] = df_merged['Total_Sales'] / df_merged['Total_Demand_5min']
df_merged['AR_10min'] = df_merged['Total_Sales'] / df_merged['Total_Demand_10min']
df_merged['AR_15min'] = df_merged['Total_Sales'] / df_merged['Total_Demand_15min']

# Clean the rest
df_merged = df_merged.fillna(0)
df_merged = df_merged.sort_values(['grid_name'])  # Make sure to use the correct column name
df_merged = df_merged.round(2)

print(df_merged)

#The supply-demand ration for P1 is 1.09 and the supply-demand ration for P2 is 1.2

   grid_name  Total_Sales POI_Name  Total_Demand_5min  Total_Demand_10min  \
0          1            4     POI1               3.67                3.67   
1          2            4     POI2               3.33                3.33   

   Total_Demand_15min  AR_5min  AR_10min  AR_15min  
0                3.67     1.09      1.09      1.09  
1                3.33     1.20      1.20      1.20  


# Calculate the accessibility per grid

In [90]:
# Step 1: Remove last column using .iloc (pandas-safe)
Dist5_trimmed = Dist5.iloc[:, :-1]
Dist10_trimmed = Dist10.iloc[:, :-1]
Dist15_trimmed = Dist15.iloc[:, :-1]

# Step 2: Matrix multiplication (convert to NumPy)
Accessibility_5min = np.matmul(Dist5_trimmed.values, df_merged['AR_5min'].values)
Accessibility_10min = np.matmul(Dist10_trimmed.values, df_merged['AR_10min'].values)
Accessibility_15min = np.matmul(Dist15_trimmed.values, df_merged['AR_15min'].values)

# Step 3: Stack and format output
Accessibility = pd.DataFrame(
    np.vstack([Accessibility_5min, Accessibility_10min, Accessibility_15min]).T,
    columns=['Accessibility_5min', 'Accessibility_10min', 'Accessibility_15min']
)

# Step 4: Round and print
Accessibility = Accessibility.round(2)
print("✅ Updated Accessibility DataFrame:")
print(Accessibility)
#The accessibility for grid v5 which in the ordered set V={v2,v7,v1,v3,v4,v5,v6,v8,v9} is grid_name 6 is 2.29


✅ Updated Accessibility DataFrame:
   Accessibility_5min  Accessibility_10min  Accessibility_15min
0                1.09                 1.09                 1.09
1                1.20                 1.20                 1.20
2                0.00                 0.00                 0.00
3                1.09                 1.09                 1.09
4                1.20                 1.20                 1.20
5                2.29                 2.29                 2.29
6                1.09                 1.09                 1.09
7                1.20                 1.20                 1.20
8                0.00                 0.00                 0.00


Create the pivot view for maps

In [92]:
# Fix Metric_Type naming
df_long['Metric_Type'] = df_long['Metric'].str.extract(r'([A-Za-z]+)').squeeze()

# Pivot again
df_pivoted = df_long.pivot_table(
    index=['grid_name', 'POI_Name', 'Total_Sales', 'Drive_Time_Ring'],
    columns='Metric_Type',
    values='Value'
).reset_index()

# Rename if needed
df_pivoted = df_pivoted.rename(columns={'Total': 'Total_Demand'})

# Final column order
df_pivoted = df_pivoted[['POI_Name', 'grid_name', 'Total_Sales', 'Total_Demand', 'AR', 'Drive_Time_Ring']]
df_pivoted = df_pivoted.round(6)

print(df_pivoted)

Metric_Type POI_Name  grid_name  Total_Sales  Total_Demand    AR  \
0               POI1          1            4          3.67  1.09   
1               POI1          1            4          3.67  1.09   
2               POI1          1            4          3.67  1.09   
3               POI2          2            4          3.33  1.20   
4               POI2          2            4          3.33  1.20   
5               POI2          2            4          3.33  1.20   

Metric_Type  Drive_Time_Ring  
0                          5  
1                         10  
2                         15  
3                          5  
4                         10  
5                         15  
